<hr>
<br>

#### - **prepare_data**.py : 데이터 다운로드 & 데이터 로더 생성을 위한 코드
<br>

#### - **train_funcs**.py : 모델 학습과 관련된 함수들을 위한 코드
<br>

#### - **build_model**.py : 모델 생성을 위한 코드
<br>

#### - **train**.py : 모델 학습을 위한 코드
<br>

#### - **utils**.py : Utility 함수들을 위한 코드
<br>

<hr>

In [1]:
# 자동완성을 위한 import 

import os
import zipfile
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets, transforms

from torchmetrics import Accuracy # PyTorch metrics

/opt/anaconda3/envs/goormthon/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<br>
<br>

# 1.  prepare_data.py

<br>

### -> 데이터 다운로드 & 데이터 로더 생성을 위한 코드

In [20]:
%%writefile model_module/prepare_data.py 
# "Write a file" with the following source-codes


import os
import zipfile

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader # class for loading the dataset (Dataset -> Batches)


def create_dataloaders(train_dir, 
                       test_dir, 
                       train_transform, 
                       test_transform,
                       batch_size):

    with zipfile.ZipFile("data_food101/Food101.zip", "r") as zip_f:
        print("Unzipping the dataset.") 
        zip_f.extractall("data_food101/data") # "extract" "all" files 

        
    # 1) image directories -> image transformation -> ImageFolder
    
    train_imgfolder = datasets.ImageFolder(root=train_dir, transform=train_transform)
    test_imgfolder  = datasets.ImageFolder(root=test_dir,  transform=train_transform)

    class_names = train_imgfolder.classes

    
    # 2) ImageFolder -> DataLoader (iterable for mini-batches)
    
    torch.manual_seed(42) # "Manually" set the "seed" 
    
    train_dataloader = DataLoader(train_imgfolder,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  pin_memory=True # GPU로의 데이터 로딩 속도 향상 @ https://bit.ly/3B70xIV
    )
    test_dataloader = DataLoader(test_imgfolder,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 pin_memory=True # GPU로의 데이터 로딩 속도 향상 @ https://bit.ly/3B70xIV
    )
    
    return train_dataloader, test_dataloader, class_names

Overwriting model_module/prepare_data.py


<br>
<br>

# 2.  build_model.py

<br>

### -> 모델 생성을 위한 코드

In [26]:
%%writefile model_module/build_model.py


import torch
from torch import nn # 'N'eural 'N'etworks 


class CNNAugment_TinyVGG(nn.Module):

    
    def __init__(self, num_channels, num_filters, num_classes): 
        
        super().__init__()
        
        self.conv_block_entrance = nn.Sequential(
            
            nn.Conv2d(in_channels=num_channels, # will be '3' == R/G/B
                      out_channels=num_filters, # num_filters == num of feature-maps == num of output channels
                      kernel_size=(3, 3), 
                      stride=1,                 # default
                      padding=1),               # 0 == 'valid', 1 == 'same'
            nn.BatchNorm2d(num_filters),        # Batch-normalization on 2-Dimensional data
            nn.ReLU(),
            
            nn.Conv2d(in_channels=num_filters,  # should be same as the number of "channels of previous output"
                      out_channels=num_filters,
                      kernel_size=(3, 3),
                      stride=1,
                      padding=1),
            nn.BatchNorm2d(num_filters),        # Batch-normalization 
            nn.ReLU(),
            
            nn.MaxPool2d(kernel_size=2, stride=2) # Default == kernel_size (자동으로 지정됨)
        )
        # [ 32, 3, 64, 64 ] -> [ 32, 10, 32, 32 ]
        
        
        self.conv_block_hidden_1 = nn.Sequential(
            nn.Conv2d(num_filters, num_filters, (3, 3), padding=1),
            nn.BatchNorm2d(num_filters), # Batch-normalization 
            nn.ReLU(),
            nn.Conv2d(num_filters, num_filters, (3, 3), padding=1),
            nn.BatchNorm2d(num_filters), # Batch-normalization 
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # [ 32, 10, 32, 32 ] -> [ 32, 10, 16, 16 ]
        
        
        self.conv_block_hidden_2 = nn.Sequential(
            nn.Conv2d(num_filters, num_filters, (3, 3), padding=1),
            nn.BatchNorm2d(num_filters), # Batch-normalization 
            nn.ReLU(),
            nn.Conv2d(num_filters, num_filters, (3, 3), padding=1),
            nn.BatchNorm2d(num_filters), # Batch-normalization 
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # [ 32, 10, 16, 16 ] -> [ 32, 10, 8, 8 ]
        
        
        self.classifier_block = nn.Sequential(
            nn.Flatten(), # Flatten the input data
            nn.Dropout(0.5), # Drop-out
            nn.Linear(in_features=num_filters * 8 * 8, 
                      out_features=num_filters * 16 * 16),
            nn.ReLU(),
            nn.Dropout(0.5), # Drop-out
            nn.Linear(in_features=num_filters * 16 * 16, 
                      out_features=num_classes)
        )
        # [ 32, 10, 8, 8 ] -> [ 32, 10 * 8 * 8 ] -> [ 32, 10 * 16 * 16 ] -> [ 32, 10 ]
    
    
    def forward(self, x):
        
        x = self.conv_block_enterance(x)
        # print(x.shape)
        
        x = self.conv_block_hidden_1(x)
        # print(x.shape)
        
        x = self.conv_block_hidden_2(x)
        # print(x.shape)
        
        x = self.classifier_block(x)
        # print(x.shape)
        
        # 아래와 같이 코드를 작성하게되면 메모리가 크게 소요되는 변수 재할당 과정이 생략되므로 계산 속도가 향상됩니다. (https://bit.ly/3V16ZJy)
        # return self.classifier_block(conv_block_hidden_2(conv_block_hidden_1(conv_block_entrance(x)))
        
        return x

Overwriting model_module/build_model.py


<br>
<br>

# 3. train_funcs.py

<br>

### -> 모델 학습과 관련된 함수들을 위한 코드

In [5]:
%%writefile model_module/train_funcs.py


import torch
from tqdm.auto import tqdm # we will use tqdm


def train_step(model, dataloader, loss_fn, optimizer, metric, device):
    
    # 모델을 training mode로 설정 (default state)
    model.train()
    
    # train-loss & train-accuracy for one epoch
    train_loss = 0
    train_acc  = 0
    
    for batch_idx, (X, y) in enumerate(dataloader): # X & y == a single batch
        
        X = X.to(device)
        y = y.to(device)
        
        # 1. (x 데이터를 모델에 넣고) 순방향 계산 진행 (forward pass)
        logits = model(X)

        # 2. (Batch) Training cost 계산 (Cost function 계산)
        loss = loss_fn(logits, y) # cost of batch <- nn.CrossEntropyLoss() : built-in Softmax
        train_loss += loss.item()
        
        # 3. Optimizer 내부의 이전 gradient 값 초기화 (Make "grad" to "zero")
        optimizer.zero_grad()

        # 4. Back-propagation ("Backward" propagation)
        loss.backward()

        # 5. Gradient descent 진행 (Take a "step" to update parameters)
        optimizer.step()

        # 6. (Batch) Training accuracy 계산 
        predicted_classes = logits.softmax(dim=1).argmax(dim=1)
        train_acc += metric(predicted_classes, y).item() # calculate the batch accuracy & add to the epoch accuracy

    # Batch 순회 종료 후
    train_loss = train_loss / len(dataloader) # cost of batches / num of batches (calculate average)
    train_acc  = train_acc  / len(dataloader) # acc  of batches / num of batches (calculate average)
    
    return train_loss, train_acc


def test_step(model, dataloader, loss_fn, metric, device):
    
    # 모델을 evaluation mode로 설정
    model.eval() 
    
    # test-loss & test-accuracy for one epoch
    test_loss = 0
    test_acc  = 0
    
    with torch.inference_mode(): # Set "inference mode"
        
        for batch_idx, (X, y) in enumerate(dataloader): # X & y == a single batch
            
            X = X.to(device)
            y = y.to(device)
    
            # 1. (x 데이터를 모델에 넣고) 순방향 계산 진행 (forward pass)
            logits = model(X)

            # 2. (Batch) Test cost 계산 (Cost function 계산)
            loss = loss_fn(logits, y) # cost of batch <- nn.CrossEntropyLoss() : built-in Softmax
            test_loss += loss.item()

            # 3. (Batch) Test accuracy 계산 
            predicted_classes = logits.softmax(dim=1).argmax(dim=1)
            test_acc += metric(predicted_classes, y).item() # calculate the batch accuracy & add to the epoch accuracy

    
    # Batch 순회 종료 후
    test_loss = test_loss / len(dataloader) # cost of batches / num of batches (calculate average)
    test_acc  = test_acc  / len(dataloader) # acc  of batches / num of batches (calculate average)
    
    return test_loss, test_acc


def train(model, 
          train_dataloader, 
          test_dataloader, 
          optimizer, 
          loss_fn, 
          metric, 
          device, 
          epochs):
    
    results = {"train_loss": [], 
               "train_acc" : [], 
               "test_loss" : [], 
               "test_acc"  : []}
    
    for epoch in tqdm(range(epochs)): # from tqdm.auto import tqdm
        
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn, 
                                           optimizer=optimizer, 
                                           metric=metric, 
                                           device=device)
        
        test_loss, test_acc   = test_step(model=model,
                                          dataloader=test_dataloader, 
                                          loss_fn=loss_fn, 
                                          metric=metric, 
                                          device=device)
        
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        print('Epoch : {} | Train_loss : {} | Train_acc : {} | Test_loss : {} | Test_acc : {}'.format(epoch+1, 
                                                                                                      train_loss, 
                                                                                                      train_acc, 
                                                                                                      test_loss, 
                                                                                                      test_acc))
    return results

Writing model_module/train_funcs.py


<br>
<br>

# 4.  utils.py

<br>

### -> Utility 함수들을 위한 코드

In [6]:
%%writefile model_module/utils.py


import torch

def save_model(model, 
               target_dir,
               model_name):

    model_save_path = target_dir + '/' + model_name
    
    print("[INFO] Saving model to: {}".format(model_save_path))
    
    torch.save(obj=model.state_dict(),
               f=model_save_path)

Writing model_module/utils.py


<br>
<br>

# 5.  train.py

<br>

### -> 모델 학습을 위한 코드

<br>

### * How to use [ **argparse** ]

In [7]:
import argparse

parser = argparse.ArgumentParser()

parser.pars

AttributeError: 'ArgumentParser' object has no attribute 'pars'

In [8]:
%%writefile model_module/argparser_test.py

import argparse # "arg"uments "parse"r


parser = argparse.ArgumentParser(description="Test for the argparser")

# "test_number"라는 이름의 argument를 받아내도록 세팅 <- default 30 & 'int' type 
parser.add_argument("--test_number", 
                     default=30, 
                     type=int, 
                     help="set any integer to use")

args = parser.parse_args()

TEST_NUM = args.test_number # "--test_number"

print('Selected number is {}'.format(TEST_NUM))

Writing model_module/argparser_test.py


In [10]:
!python model_module/argparser_test.py --help

# -h or --help

usage: argparser_test.py [-h] [--test_number TEST_NUMBER]

Test for the argparser

optional arguments:
  -h, --help            show this help message and exit
  --test_number TEST_NUMBER
                        set any integer to use


In [11]:
!python model_module/argparser_test.py --test_number 42

Selected number is 42


In [16]:
%%writefile model_module/train.py


import os
import argparse # "arg"uments "parse"r

import torch
from torchvision import transforms
from torchmetrics import Accuracy # PyTorch metrics

import prepare_data, train_funcs, build_model, utils


parser = argparse.ArgumentParser(description="Argparser for hyper-parameters")

parser.add_argument("--num_epochs", 
                     default=30, 
                     type=int, 
                     help="the number of epochs")

parser.add_argument("--batch_size",
                    default=32,
                    type=int,
                    help="number of samples per batch")

parser.add_argument("--num_filters",
                    default=32,
                    type=int,
                    help="number of filters to use in convolution layers")

parser.add_argument("--learning_rate",
                    default=0.001,
                    type=float, # set the data type of the argument
                    help="learning-rate")

parser.add_argument("--train_dir",
                    default="data_food101/data/train",
                    type=str, # set the data type of the argument
                    help="directory path of training data")

parser.add_argument("--test_dir",
                    default="data_food101/data/test",
                    type=str, # set the data type of the argument
                    help="directory path of testing data")


args = parser.parse_args()

NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
NUM_FILTERS = args.num_filters
LEARNING_RATE = args.learning_rate

print("[INFO] Setup - Epochs : {} | Batch_size : {} | Num_filters : {} | Learning_rate : {}".format(NUM_EPOCHS, 
                                                                                                    BATCH_SIZE, 
                                                                                                    NUM_FILTERS, 
                                                                                                    LEARNING_RATE))

train_dir = args.train_dir
test_dir = args.test_dir

print("[INFO] Training directory : {}".format(train_dir))
print("[INFO] Testing directory : {}".format(test_dir))


device = "mps" if torch.backends.mps.is_available() else "cpu"


train_transform = transforms.Compose([
    transforms.Resize((64, 64)), # image resize
    transforms.TrivialAugmentWide(num_magnitude_bins=31), # TrivialAugment
    transforms.ToTensor() # (Original) PIL format -> PyTorch tensors 
])
test_transform = transforms.Compose([
    transforms.Resize((64, 64)), # image resize
    transforms.ToTensor() # (Original) PIL format -> PyTorch tensors 
])

train_dataloader, test_dataloader, class_names = prepare_data.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transform=train_transform,
    test_transform=test_transform,
    batch_size=BATCH_SIZE
)


model = build_model.CNNAugment_TinyVGG(num_channels=3, 
                                       num_filters=NUM_FILTERS, 
                                       num_classes=len(class_names)).to(device)

loss_fn = torch.nn.CrossEntropyLoss() # Softmax + CrossEntropy (built-in Softmax)

optimizer = torch.optim.Adam(params=model.parameters(), # "parameters" to optimize (apply gradient descent)
                             lr=LEARNING_RATE)                  # "l"earning "r"ate 
    
metric_accuracy = Accuracy(task='multiclass',
                            num_classes=3).to(device) # from torchmetrics import Accuracy

    
train_funcs.train(model=model,
                  train_dataloader=train_dataloader,
                  test_dataloader=test_dataloader,
                  optimizer=optimizer,
                  loss_fn=loss_fn,
                  metric=metric_accuracy,
                  device=device,
                  epochs=NUM_EPOCHS)


utils.save_model(model=model,
                 target_dir="model_module/models",
                 model_name="CNNAugment_TinyVGG_modular.pth")

Overwriting model_module/train.py


In [36]:
# !python model_module/train.py --help

In [27]:
!python model_module/train.py --num_epochs 1 --batch_size 32 --num_filters 32 --learning_rate 0.001

# 종료 시점이 아닌 실시간으로 실행 결과를 확인하고 싶을 경우 cmd 창에서 실행 -> 10 epochs 이상 학습 진행 후 다음 cell의 이미지 예측 진행

[INFO] Setup - Epochs : 1 | Batch_size : 32 | Num_filters : 32 | Learning_rate : 0.001
[INFO] Training directory : data_food101/data/train
[INFO] Testing directory : data_food101/data/test
Unzipping the dataset.
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.42s/it]
[INFO] Saving model to: model_module/models/CNNAugment_TinyVGG_modular.pth


<br>
<br>

# (Appendix)  predict_img.py

<br>

### -> 특정 이미지를 대상으로 한 Prediction을 위한 코드

In [122]:
%%writefile model_module/predict.py


import argparse

import torch
import torchvision

import ? # to build a model


parser = argparse.Arg?()

parser.add_?("--image_path",
                    help="path of the image to predict on")

parser.add_?("--model_path",
                    default="model_module/models/CNNAugment_TinyVGG_modular.pth",
                    type=str,
                    help="path of the saved model-parameters (.pth)")


args = parser.parse_?()

device = "cuda" if torch.cuda.is_available() else "cpu"


IMG_PATH = args.?
print("[INFO] Predicting on {}".format(IMG_PATH))


def load_model(file_path=args.?):
    
    model = build_model.?(num_channels=3, 
                                             num_filters=32, 
                                             num_classes=3).to(device)

    print("[INFO] Loading model-parameters from: {}".format(file_path))

    model.?(torch.?(file_path)) # load -> load_state_dict

    return model


def predict_on_image(image_path=?, file_path=args.?):

    model = ?(file_path) # Load the model

    img_tensor = torchvision.io.read_?(IMG_PATH) # instead of [ ImageFolder & DataLoader ]
    
    img_tensor = img_tensor / 255.0
    img_tensor = img_tensor.?(dim=0) # [Channels, Height, Width] -> [Batch_size, Channels, Height, Width]
    
    transform = torchvision.transforms.?(size=(64, 64)) # Don't need 'transforms.Compose'
    img_tensor_transformed = transform(img_tensor) 

    
    model.?()
    
    with torch.?():
        
        pred_logits = ?(img_tensor_transformed.?(device)) 
        
        pred_prob = pred_logits.?(dim=1)
        pred_prob_top = pred_prob.max(dim=1)[0].item()
        
        pred_label = pred_prob.?(dim=1)
        pred_label_class = ["pizza", "steak", "sushi"][?]

        print("[INFO] Predicted class: {}, Prediction probability: {:.3f}".format(pred_label_class,
                                                                                  pred_prob_top))
        
if __name__ == "__main__":
    
    ?()

Overwriting model_module/predict.py


In [123]:
!python model_module/predict.py --? data_food101/sample_pizza.jpg

[INFO] Predicting on data_food101/sample_pizza.jpg
[INFO] Loading model-parameters from: model_module/models/CNNAugment_TinyVGG_modular.pth
[INFO] Predicted class: sushi, Prediction probability: 0.493
